This script contains utility code to clean up certain things

In [ ]:
# Cleans up spectro dataset - removes all .spec files that does not contain any valid data
from remucs.util import SpectrogramCollection
import os
from tqdm.auto import tqdm
from scripts.calculate import DATASET_PATH
from remucs.dataset import get_valid_bar_numbers

deletable = []
for file in tqdm(os.listdir(DATASET_PATH), desc="Checking files..."):
    if file.endswith(".spec.zip"):
        try:
            spec = SpectrogramCollection.load(os.path.join(DATASET_PATH, file))
            valids = get_valid_bar_numbers(list(spec.spectrograms.keys()))
            if len(valids) == 0:
                deletable.append(file)
        except Exception as e:
            print(f"Error loading {file}: {e}")
            deletable.append(file)

# Removes the files
for line in deletable:
    os.remove(os.path.join(DATASET_PATH, line.strip()))

In [ ]:
# Some spectrograms have a peculiar white border, remove those data points
from remucs.util import SpectrogramCollection
import os
from tqdm.auto import tqdm
import torch

SPECTROGRAM_SAVE_PATH = r"D:\Repository\project-remucs\audio-infos-v3\test_specs"

for file in tqdm(os.listdir(SPECTROGRAM_SAVE_PATH), desc="Checking files..."):
    if file.endswith(".spec.zip"):
        try:
            modified = False
            specs = SpectrogramCollection.load(os.path.join(SPECTROGRAM_SAVE_PATH, file))
            s = list(specs.spectrograms.keys())
            for part, bar in s:
                spec = specs.get_spectrogram(part, bar)
                if spec is not None:
                    # Check if there is any row of 1s
                    if torch.any(torch.all(spec == 0, dim = -1)).item():
                        specs.spectrograms.pop((part, bar))
                        modified = True
            if modified:
                specs.save(os.path.join(SPECTROGRAM_SAVE_PATH, file))
        except Exception as e:
            print(f"Error loading {file}: {e}")